<a href="https://colab.research.google.com/github/daisysong76/Prompt-Engineering-Guide/blob/main/prompt_optermization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import React, { useState, useEffect } from 'react';
import { Play, RefreshCw, Zap, Brain, Target, TrendingUp, Star, ChevronRight } from 'lucide-react';

const PromptOptimizer = () => {
  const [task, setTask] = useState('Summarize this article in a compelling way');
  const [examples, setExamples] = useState([
    { input: 'Long technical article about AI', output: 'AI breakthrough: New model achieves human-level performance' },
    { input: 'Research paper on climate change', output: 'Study reveals urgent climate action needed by 2030' }
  ]);
  const [generatedPrompts, setGeneratedPrompts] = useState([]);
  const [evolutionHistory, setEvolutionHistory] = useState([]);
  const [isRunning, setIsRunning] = useState(false);
  const [currentGeneration, setCurrentGeneration] = useState(0);
  const [selectedMethod, setSelectedMethod] = useState('hybrid');

  // Simulated LLM completion function
  const generateWithClaude = async (prompt) => {
    // In a real implementation, this would call window.claude.complete()
    // For demo purposes, we'll simulate different response qualities
    const responses = [
      "Here's a concise summary: [Generated content]",
      "Summary: [AI-generated text with good structure]",
      "Brief overview: [Well-formatted summary]",
      "Key points: [Bullet-pointed summary]"
    ];
    
    // Simulate variable quality based on prompt characteristics
    const quality = Math.random() * 0.4 + (prompt.length > 100 ? 0.6 : 0.3);
    const responseIndex = Math.floor(Math.random() * responses.length);
    
    return {
      text: responses[responseIndex],
      quality: quality,
      metrics: {
        clarity: Math.random() * 0.3 + quality,
        specificity: Math.random() * 0.3 + quality,
        engagement: Math.random() * 0.3 + quality
      }
    };
  };

  // 1. LLM-Based Prompt Generator (APE-style)
  const generateSeedPrompts = async () => {
    const seedPrompts = [
      `You are an expert writer. ${task}. Be specific and engaging.`,
      `Task: ${task}. Focus on key insights and make it compelling for readers.`,
      `Please ${task.toLowerCase()}. Use clear language and highlight the most important points.`,
      `Your goal is to ${task.toLowerCase()}. Structure your response logically and concisely.`,
      `As a professional content creator, ${task.toLowerCase()}. Ensure clarity and impact.`
    ];
    
    // Simulate LLM generating variations
    const variations = [];
    for (let i = 0; i < 10; i++) {
      const base = seedPrompts[Math.floor(Math.random() * seedPrompts.length)];
      const modifiers = [
        " Think step by step.",
        " Consider the audience perspective.",
        " Focus on actionable insights.",
        " Use an engaging tone.",
        " Prioritize accuracy and clarity."
      ];
      const modifier = modifiers[Math.floor(Math.random() * modifiers.length)];
      variations.push(base + modifier);
    }
    
    return variations;
  };

  // 2. Evolutionary Algorithm (GAAPO/EvoPrompt)
  const mutatePrompt = (prompt) => {
    const mutations = [
      (p) => p.replace(/\.$/, ". Be concise."),
      (p) => p.replace(/You are/, "You're an expert who is"),
      (p) => p.replace(/Please/, "I need you to"),
      (p) => p + " Focus on the most important aspects.",
      (p) => p.replace(/Task:/, "Your mission:")
    ];
    
    const mutation = mutations[Math.floor(Math.random() * mutations.length)];
    return mutation(prompt);
  };

  const crossoverPrompts = (prompt1, prompt2) => {
    const words1 = prompt1.split(' ');
    const words2 = prompt2.split(' ');
    const splitPoint = Math.floor(words1.length / 2);
    
    return words1.slice(0, splitPoint).concat(words2.slice(splitPoint)).join(' ');
  };

  const evolveGeneration = async (population) => {
    // Evaluate fitness
    const evaluated = await Promise.all(
      population.map(async (prompt) => {
        const response = await generateWithClaude(prompt);
        return { prompt, fitness: response.quality, metrics: response.metrics };
      })
    );

    // Sort by fitness
    evaluated.sort((a, b) => b.fitness - a.fitness);
    
    // Select top 50% as parents
    const parents = evaluated.slice(0, Math.ceil(evaluated.length / 2));
    
    // Generate offspring through mutation and crossover
    const offspring = [];
    for (let i = 0; i < population.length - parents.length; i++) {
      if (Math.random() < 0.7) {
        // Mutation
        const parent = parents[Math.floor(Math.random() * parents.length)];
        offspring.push(mutatePrompt(parent.prompt));
      } else {
        // Crossover
        const parent1 = parents[Math.floor(Math.random() * parents.length)];
        const parent2 = parents[Math.floor(Math.random() * parents.length)];
        offspring.push(crossoverPrompts(parent1.prompt, parent2.prompt));
      }
    }
    
    return {
      newPopulation: [...parents.map(p => p.prompt), ...offspring],
      bestFitness: evaluated[0].fitness,
      avgFitness: evaluated.reduce((sum, p) => sum + p.fitness, 0) / evaluated.length,
      bestPrompt: evaluated[0]
    };
  };

  // 3. AutoHint Generation
  const generateHints = (failurePrompts) => {
    const hints = [
      "Be more specific in your instructions",
      "Add context about the target audience",
      "Include formatting guidelines",
      "Specify the desired length or style",
      "Add examples of good outputs"
    ];
    
    return hints.slice(0, Math.floor(Math.random() * 3) + 1);
  };

  // 4. Retrieval-Augmented Prompting
  const retrieveSimilarPrompts = (task) => {
    const promptLibrary = [
      "Summarize this content for a general audience, focusing on key takeaways.",
      "Create an engaging summary that captures the main points and implications.",
      "Write a concise overview highlighting the most important information."
    ];
    
    return promptLibrary.filter(p =>
      p.toLowerCase().includes(task.toLowerCase().split(' ')[0])
    );
  };

  // Main optimization pipeline
  const runOptimization = async () => {
    setIsRunning(true);
    setGeneratedPrompts([]);
    setEvolutionHistory([]);
    setCurrentGeneration(0);

    try {
      // Stage 1: Generate seed prompts
      let population = await generateSeedPrompts();
      
      // Stage 2: Add retrieval-augmented prompts
      const retrieved = retrieveSimilarPrompts(task);
      population = [...population, ...retrieved];
      
      // Stage 3: Evolutionary optimization
      for (let gen = 0; gen < 5; gen++) {
        setCurrentGeneration(gen + 1);
        
        const result = await evolveGeneration(population);
        population = result.newPopulation;
        
        setEvolutionHistory(prev => [...prev, {
          generation: gen + 1,
          bestFitness: result.bestFitness,
          avgFitness: result.avgFitness,
          bestPrompt: result.bestPrompt
        }]);
        
        // Small delay for animation
        await new Promise(resolve => setTimeout(resolve, 500));
      }
      
      // Final evaluation and ranking
      const finalEvaluation = await Promise.all(
        population.slice(0, 8).map(async (prompt) => {
          const response = await generateWithClaude(prompt);
          return {
            prompt,
            score: response.quality,
            metrics: response.metrics,
            response: response.text
          };
        })
      );
      
      finalEvaluation.sort((a, b) => b.score - a.score);
      setGeneratedPrompts(finalEvaluation);
      
    } catch (error) {
      console.error('Optimization failed:', error);
    } finally {
      setIsRunning(false);
    }
  };

  const MethodCard = ({ title, description, icon: Icon, active, onClick }) => (
    <div
      className={`p-4 rounded-lg border cursor-pointer transition-all ${
        active ? 'border-blue-500 bg-blue-50' : 'border-gray-200 hover:border-gray-300'
      }`}
      onClick={onClick}
    >
      <div className="flex items-center gap-3 mb-2">
        <Icon className="w-5 h-5 text-blue-600" />
        <h3 className="font-semibold">{title}</h3>
      </div>
      <p className="text-sm text-gray-600">{description}</p>
    </div>
  );

  return (
    <div className="max-w-6xl mx-auto p-6 bg-gray-50 min-h-screen">
      <div className="bg-white rounded-xl shadow-lg p-8">
        <div className="mb-8">
          <h1 className="text-3xl font-bold text-gray-900 mb-2">
            🚀 Advanced Prompt Optimizer
          </h1>
          <p className="text-gray-600">
            State-of-the-art automated prompt generation using LLM-driven evolution,
            retrieval augmentation, and multi-agent optimization.
          </p>
        </div>

        {/* Configuration Section */}
        <div className="grid grid-cols-1 lg:grid-cols-2 gap-8 mb-8">
          <div>
            <label className="block text-sm font-medium text-gray-700 mb-2">
              Task Description
            </label>
            <textarea
              value={task}
              onChange={(e) => setTask(e.target.value)}
              className="w-full h-24 p-3 border border-gray-300 rounded-lg focus:ring-2 focus:ring-blue-500 focus:border-blue-500"
              placeholder="Describe what you want the prompt to accomplish..."
            />
          </div>
          
          <div>
            <label className="block text-sm font-medium text-gray-700 mb-2">
              Example Input/Output Pairs
            </label>
            <div className="space-y-2 max-h-24 overflow-y-auto">
              {examples.map((ex, idx) => (
                <div key={idx} className="text-xs p-2 bg-gray-50 rounded border">
                  <div><strong>Input:</strong> {ex.input}</div>
                  <div><strong>Output:</strong> {ex.output}</div>
                </div>
              ))}
            </div>
          </div>
        </div>

        {/* Method Selection */}
        <div className="mb-8">
          <h2 className="text-xl font-semibold mb-4">Optimization Methods</h2>
          <div className="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-4 gap-4">
            <MethodCard
              title="APE Generator"
              description="LLM-based prompt generation from examples"
              icon={Brain}
              active={selectedMethod.includes('ape')}
              onClick={() => setSelectedMethod('ape')}
            />
            <MethodCard
              title="GAAPO Evolution"
              description="Genetic algorithm with mutation & crossover"
              icon={RefreshCw}
              active={selectedMethod.includes('evolution')}
              onClick={() => setSelectedMethod('evolution')}
            />
            <MethodCard
              title="RAG Enhancement"
              description="Retrieval-augmented prompt optimization"
              icon={Target}
              active={selectedMethod.includes('rag')}
              onClick={() => setSelectedMethod('rag')}
            />
            <MethodCard
              title="Hybrid Pipeline"
              description="Combined multi-stage optimization"
              icon={Zap}
              active={selectedMethod === 'hybrid'}
              onClick={() => setSelectedMethod('hybrid')}
            />
          </div>
        </div>

        {/* Control Panel */}
        <div className="flex items-center justify-between mb-8 p-4 bg-gray-50 rounded-lg">
          <div className="flex items-center gap-4">
            <button
              onClick={runOptimization}
              disabled={isRunning}
              className="flex items-center gap-2 px-6 py-3 bg-blue-600 text-white rounded-lg hover:bg-blue-700 disabled:bg-gray-400 transition-colors"
            >
              {isRunning ? (
                <>
                  <RefreshCw className="w-5 h-5 animate-spin" />
                  Optimizing...
                </>
              ) : (
                <>
                  <Play className="w-5 h-5" />
                  Start Optimization
                </>
              )}
            </button>
            
            {isRunning && (
              <div className="text-sm text-gray-600">
                Generation {currentGeneration}/5 - Evolving prompts...
              </div>
            )}
          </div>
          
          {evolutionHistory.length > 0 && (
            <div className="text-sm text-gray-600">
              Best Score: {(evolutionHistory[evolutionHistory.length - 1]?.bestFitness * 100).toFixed(1)}%
            </div>
          )}
        </div>

        {/* Evolution Progress */}
        {evolutionHistory.length > 0 && (
          <div className="mb-8">
            <h2 className="text-xl font-semibold mb-4 flex items-center gap-2">
              <TrendingUp className="w-5 h-5" />
              Evolution Progress
            </h2>
            <div className="bg-white border rounded-lg p-4">
              <div className="flex items-end gap-2 h-32">
                {evolutionHistory.map((gen, idx) => (
                  <div key={idx} className="flex-1 flex flex-col items-center">
                    <div
                      className="w-full bg-blue-500 rounded-t"
                      style={{ height: `${gen.bestFitness * 100}%` }}
                    />
                    <div className="text-xs mt-1">Gen {gen.generation}</div>
                  </div>
                ))}
              </div>
            </div>
          </div>
        )}

        {/* Results */}
        {generatedPrompts.length > 0 && (
          <div>
            <h2 className="text-xl font-semibold mb-4 flex items-center gap-2">
              <Star className="w-5 h-5" />
              Optimized Prompts (Ranked by Performance)
            </h2>
            <div className="space-y-4">
              {generatedPrompts.map((item, idx) => (
                <div key={idx} className="border rounded-lg p-4 hover:shadow-md transition-shadow">
                  <div className="flex items-start justify-between mb-3">
                    <div className="flex items-center gap-2">
                      <span className="text-lg font-bold text-blue-600">#{idx + 1}</span>
                      <div className="flex items-center gap-4 text-sm text-gray-600">
                        <span>Score: {(item.score * 100).toFixed(1)}%</span>
                        <span>Clarity: {(item.metrics.clarity * 100).toFixed(0)}%</span>
                        <span>Engagement: {(item.metrics.engagement * 100).toFixed(0)}%</span>
                      </div>
                    </div>
                    <ChevronRight className="w-5 h-5 text-gray-400" />
                  </div>
                  
                  <div className="mb-3">
                    <h4 className="font-medium text-gray-900 mb-1">Optimized Prompt:</h4>
                    <p className="text-gray-700 bg-gray-50 p-3 rounded border">
                      {item.prompt}
                    </p>
                  </div>
                  
                  <div>
                    <h4 className="font-medium text-gray-900 mb-1">Sample Output:</h4>
                    <p className="text-gray-600 text-sm italic">
                      {item.response}
                    </p>
                  </div>
                </div>
              ))}
            </div>
          </div>
        )}

        {/* Technical Notes */}
        <div className="mt-8 p-4 bg-blue-50 rounded-lg border border-blue-200">
          <h3 className="font-semibold text-blue-900 mb-2">Implementation Notes</h3>
          <ul className="text-sm text-blue-800 space-y-1">
            <li>• This prototype simulates the full pipeline with mock LLM responses</li>
            <li>• In production, replace generateWithClaude() with actual window.claude.complete() calls</li>
            <li>• Add more sophisticated fitness functions based on your specific metrics</li>
            <li>• Scale the population size and generations based on computational budget</li>
            <li>• Implement proper prompt templating and hint injection mechanisms</li>
          </ul>
        </div>
      </div>
    </div>
  );
};

export default PromptOptimizer;

https://claude.ai/public/artifacts/cde2ebde-4e08-43b6-ad5f-d63d6e8ae6ab
